This tutorial can be downloaded [link](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic/basic_004.ipynb).

# 4.0 Plotting density of states and local density of states

We show how to use the [WESTpy Python package](http://www.west-code.org/doc/westpy/latest/) to plot the density of states and local density of states from GW quasiparticle energy levels calculated using WEST.

## Running DFT and GW calculations

We will use the negatively charged nitrogen-vacancy center in diamond as an example. Download the input and pseudopotential files to your current directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond/pw.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond/wstat.in
wget -N -q http://www.west-code.org/doc/training/nv_diamond/wfreq.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.2.upf

Let's inspect the ``pw.in`` file.

In [ ]:
%%bash
cat pw.in

This is a relatively big system with 215 atoms and 862 valence electrons, and requires an explicit treatment of spin polarization. As detailed in [Yu et al., J. Chem. Theory Comput. 18, 4690-4707 (2022)](https://doi.org/10.1021/acs.jctc.2c00241), WEST can use GPU accelerators to carry out large-scale GW calculations. For instance, we can use four GPUs for the ground state DFT calculation, and 64 GPUs for the GW calculation.

In [ ]:
%%bash
mpirun -n 4 pw.x -i pw.in > pw.out
mpirun -n 64 wstat.x -nimage 16 -npool 2 -nbgrp 2 -i wstat.in > wstat.out
mpirun -n 64 wfreq.x -nimage 8 -npool 2 -nbgrp 4 -i wfreq.in > wfreq.out

Note that we have used the image, pool, and band group parallelization levels, as discussed in the [3.0 Tutorial](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/tutorials/basic_003.ipynb). By using all the parallelization levels, we avoid carrying out fast Fourier transforms (FFTs) on multiple GPUs, which is inefficient because of the all-to-all communication involved in parallel FFTs.

If the reader does not have the computational resource to run the calculation, the WEST output file needed for the next step can be directly downloaded:

In [ ]:
%%bash
mkdir nv_diamond.wfreq.save
wget -N -q http://www.west-code.org/doc/training/nv_diamond/wfreq.json -O nv_diamond.wfreq.save/wfreq.json

## Plotting density of states

The density of states (DOS) is defined as follows:
\begin{equation}
\text{DOS}(E) = \frac{1}{V} \sum_{i=1}^N \delta (E-E_i)
\end{equation}
where $E_i$ is the energy of the $i$th wavefunction obtained at the PBE or GW level of theory, $V$ is the volume of the simulation cell, and $\delta$ is the Dirac delta function. In practice, we model the delta function by a Gaussian function with a chosen width. Here we are going to use 0.1 eV.

To plot the DOS, we first load the Kohn-Sham and quasiparticle energy levels from the WEST log file in JSON format.

In [ ]:
import json

vbm_dft = 13.0777
vbm_gw = 12.4665

# read data from JSON file
with open('nv_diamond.wfreq.save/wfreq.json') as file:
    data = json.load(file)
    
# extracting energy levels from the data 
y = {}
y['dft'] = {}
y['gw'] = {}

for s in [1,2] :
    y['dft'][s] = [e-vbm_dft for e in data['output']['Q'][f'K{s:06d}']['eks']]
    y['gw'][s] = [e-vbm_gw for e in data['output']['Q'][f'K{s:06d}']['eqpSec']]

We then create an ``ElectronicStructure`` object and attach to it the DFT and GW energy levels. Finally we call the ``plotDOS`` method to plot the DOS.

In [ ]:
from westpy import *

es = ElectronicStructure()

for key in ['dft','gw'] :
    es.addKey(key,key)
    # spin loop
    for s in [1,2] :
        b = 1
        # band loop
        for energy in y[key][s] :
            es.addDataPoint([1,s,b],key,energy)
            b += 1

es.plotDOS(kk=[1],ss=[1,2],energyKeys=['dft'],energyRange=[-4.,6.,0.01],fname='dos_dft.jpg')
es.plotDOS(kk=[1],ss=[1,2],energyKeys=['gw'],energyRange=[-4.,6.,0.01],fname='dos_gw.jpg')

## Plotting local density of states

The local density of states (LDOS) is defined as follows:
\begin{equation}
\text{LDOS}(z,E) = \sum_{i=1}^N \int \frac{\text{d}x}{L_x} \int \frac{\text{d}y}{L_y} \vert \psi_i (x,y,z)\vert ^2 \delta(E-E_i)
\end{equation}
where $\psi_i$ is the $i$th wavefunction obtained at the PBE level of theory, with energy $E_i$ obtained at the PBE or GW level of theory. $L_x$ and $L_y$ are the lengths of the x and y axes of the simulation box, respectively, whereas $z$ corresponds to the z axis of the simulation box. $\delta$ is the Dirac delta function, modeled by a Gaussian function with a width of 0.05 eV in our case.

To plot the LDOS along the z axis of the simulation box, in addition to the Kohn-Sham and quasiparticle energy levels, we need the wavefunctions averaged over z. Check out the ``westpp.x`` [input description](http://www.west-code.org/doc/West/latest/) and generate an input file for WEST called ``westpp.in``.

Download this file to your current working directory:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond/westpp.in

Let's inspect the ``westpp.in`` file, input for ``westpp.x``.

In [ ]:
%%bash
cat westpp.in

Run ``westpp.x``.

In [ ]:
%%bash
mpirun -n 4 westpp.x -i westpp.in > westpp.out

This computes the wavefunctions averaged over the z axis, and stores the results in several ``wfcKxxxxxxBxxxxxx.plavz`` files in ``nv_diamond.westpp.save``. Again, if the reader does not have the computational resource to run the calculation, the output files needed for the next step can be downloaded:

In [ ]:
%%bash
wget -N -q http://www.west-code.org/doc/training/nv_diamond/nv_diamond.westpp.save.tar.gz
tar zxvf nv_diamond.westpp.save.tar.gz

We create an ``ElectronicStructure`` object and register the energy levels and their associate wavefunction files. The energy levels are sorted into three categories, namely the bulk states, the spin up defect states, and the spin down defect states. States strongly localized around the nitrogen-vacancy center are considered to be the defect states. We call the ``plotLDOS`` method to plot the LDOS.

In [ ]:
es2 = ElectronicStructure()

# defect states
defects = [430,431,432]

keys = ['bulk','up','down']
for key in keys :
    es2.addKey(key,key)

# spin loop
for s in [1,2] :
    # band loop
    for b in range(321,481) :
        prefix = 'nv_diamond.westpp.save/'
        fileName = f'{prefix}wfcK{s:06d}B{b:06d}.plavz'
        es2.addDataPoint([1,s,b],'wfcFile',fileName)

        if b in defects :
            es2.addDataPoint([1,s,b],keys[s],y['gw'][s][b-321])
        else :
            es2.addDataPoint([1,s,b],keys[0],y['gw'][s][b-321])

es2.plotLDOS(kk=[1],ss=[1,2],energyKeys=['bulk','up','down'],sigma=0.05,energyRange=[-1.,7.,0.01],fname='ldos_gw.jpg')

We should get a plot like this:
![Image of LDOS](http://greatfire.uchicago.edu/west-public/West/raw/master/Doc/images/ldos_gw.jpg)

where the bulk states, the spin up defect states, and the spin down defect states are plotted in black, red, and blue, respectively. The reader is encouraged to adapt the script to plot the LDOS at the PBE level of theory, and compare it to tbe GW result.